In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [40]:
context_length = 16
n_embed = 4
head_size = 8

input = torch.randn((context_length, n_embed))

query = nn.Linear(n_embed, head_size)
key = nn.Linear(n_embed, head_size)
value = nn.Linear(n_embed, head_size)

q = query(input)
k = key(input)
v = value(input)
# print(q.shape) # (context_length, head_size)

# layman programmer's explanation of what mat mul achieves here
# basically taking the dot product of each key <> query pair
#
#                4     4     5     8
#                |     |     |     |
#                2     5     3     4
#                |     |     |     |
#                2     2     4     4
#                |     |     |     |
#  2---3---0----{X}---{X}---{X}-->{X}
#                |     |     |     |
#  4---5---8----{X}---{X}---{X}-->{X}
#                |     |     |     |
#  5---3---4----{X}---{X}---{X}-->{X}
#                v     v     v     v
#  2---4---4----{X}---{X}---{X}-->{X}
#
attention_matrix = []
for timestep in range(context_length):
  attention_matrix.append([])
  for other_timestep in range(context_length):
    this_query = q[timestep]
    this_key = k[other_timestep]
    dot_product = float(this_query @ this_key)
    attention_matrix[timestep].append(dot_product)

attention_matrix2 = q @ k.T

for i in range(context_length):
  for j in range(context_length):
    assert (attention_matrix[i][j] - attention_matrix2[i][j].item()) \
      < 1e-06

# once we have the attention scores, we can use them to compute
# a weighted sum of v across time for each timestep

for timestep in range(context_length):